In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import numpy as np

# Cargar un conjunto de datos de regresión como ejemplo
data = load_diabetes()
X = data.data
y = data.target

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento con XGBRegressor
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror')
xgb_reg.fit(X_train, y_train)

# Predicciones y evaluación para XGBRegressor
xgb_predictions = xgb_reg.predict(X_test)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_predictions))
print(f'RMSE de XGBRegressor: {xgb_rmse:.4f}')

# Entrenamiento con LGBMRegressor
lgb_reg = lgb.LGBMRegressor()
lgb_reg.fit(X_train, y_train)

# Predicciones y evaluación para LGBMRegressor
lgb_predictions = lgb_reg.predict(X_test)
lgb_rmse = np.sqrt(mean_squared_error(y_test, lgb_predictions))
print(f'RMSE de LGBMRegressor: {lgb_rmse:.4f}')

# Entrenamiento con CatBoostRegressor
cat_reg = CatBoostRegressor(verbose=0)  # 'verbose=0' para evitar que CatBoost imprima demasiado durante el entrenamiento
cat_reg.fit(X_train, y_train)

# Predicciones y evaluación para CatBoostRegressor
cat_predictions = cat_reg.predict(X_test)
cat_rmse = np.sqrt(mean_squared_error(y_test, cat_predictions))
print(f'RMSE de CatBoostRegressor: {cat_rmse:.4f}')


RMSE de XGBRegressor: 57.8878
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000362 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 595
[LightGBM] [Info] Number of data points in the train set: 353, number of used features: 10
[LightGBM] [Info] Start training from score 153.736544
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

In [3]:
import numpy as np
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from skopt import gp_minimize
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score

from model_info import ModelParameterExtractor
from feature_models_space import FeatureSpaceConstants
from data_transformer import DataTransformer
from model_trainer import ModelTrainer
from model_evaluator import ModelEvaluator

def mcr(model,X_train, y_train, X_eval, y_eval, eval_metric, can_we_transform_your_features):
    extractor = ModelParameterExtractor()
    model_params, model_class = extractor.get_model_params_and_class(model) #TODO: revisar
    model_type = ModelParameterExtractor.get_package_name(model_class)
    initial_feature_space = FeatureSpaceConstants.get_feature_space(model_type, can_we_transform_your_features)
    adjusted_feature_space = FeatureSpaceConstants.adjust_search_space(initial_feature_space, model_params)
    if can_we_transform_your_features:
        features_percentage_to_use_list = list([cat for cat in FeatureSpaceConstants.CONSTANTS 
                                           if cat.name == 'features_percentage_to_use'][0].categories)
        data_transformer = DataTransformer(estimator=model_class(), 
                                           X_train=X_train, 
                                           y_train=y_train, 
                                           X_eval = X_eval,
                                           percentages=features_percentage_to_use_list, 
                                           verbose=0)
    #retrained_model = model_class(**model_params)
    model_trainer = ModelTrainer(model_class, model_type)
    model_evaluator = ModelEvaluator(model_type, eval_metric)
    @use_named_args(adjusted_feature_space)
    def _objective(**kwargs):
        if can_we_transform_your_features:
            features_percentage_to_use = kwargs.pop("features_percentage_to_use")
            red_method = kwargs.pop('dimensionality_reduction')
            if red_method == 'pca':
                X_train_transformed = data_transformer.get_transformation(method='pca', percentage=features_percentage_to_use, dataset_type='train')
                X_eval_transformed = data_transformer.get_transformation(method='pca', percentage=features_percentage_to_use, dataset_type='eval')
            elif red_method == 'rfe':
                X_train_transformed = data_transformer.get_transformation( method='rfe', percentage=features_percentage_to_use, dataset_type='train')
                X_eval_transformed = data_transformer.get_transformation(method='rfe', percentage=features_percentage_to_use, dataset_type='eval')
            else:
                X_train_transformed = X_train
                X_eval_transformed = X_eval

        retrained_model = model_trainer.train_model( 
                                          X_train_transformed, 
                                          y_train, 
                                          X_eval_transformed, 
                                          y_eval, 
                                          kwargs, 
                                          eval_metric= eval_metric,
                                          early_stopping_rounds=10)

        eval, best_iter = model_evaluator.evaluate_model(retrained_model)
        return eval
    
    res = gp_minimize(_objective, adjusted_feature_space, n_calls=10, random_state=0)
    return res
    #Antes de entrenar el modelo final, si se ha aplicado alguna transformación, conseguir el transformer de la mejor ronda. ¿Cómo hacerlo?
    #final_data = data.transform(input_data)
    #final_model = usar res para entrenar el modelo final con los mejores hiperparámetros. 
    # return transformer,model (si can_we_transform_your_features = True) o model (si can_we_transform_your_features = False)

In [4]:
wtf = mcr(xgb_reg,X_train, y_train, X_train, y_train, 'rmse', True)

Adjusting gamma to new range: [0, 5]


In [5]:
best_params_dict = {dim.name: value for dim, value in zip(wtf.space, wtf.x)}

In [6]:
best_params_dict

{'max_depth': 4,
 'gamma': 4.221328742905087,
 'subsample': 0.4431782470491028,
 'colsample_bytree': 0.4389006955136503,
 'colsample_bylevel': 0.349425478714389,
 'features_percentage_to_use': 0.5,
 'dimensionality_reduction': 'pca'}

In [7]:
wtf

          fun: 17.599447548306255
            x: [4, 4.221328742905087, 0.4431782470491028, 0.4389006955136503, 0.349425478714389, 0.5, 'pca']
    func_vals: [ 1.760e+01  5.880e+01  3.651e+01  3.578e+01  5.563e+01
                 5.167e+01  5.384e+01  4.433e+01  2.290e+01  3.443e+01]
      x_iters: [[4, 4.221328742905087, 0.4431782470491028, 0.4389006955136503, 0.349425478714389, 0.5, 'pca'], [1, 1.3632814729005662, 0.29106604692854, 0.4248674915101974, 0.291990868950023, 0.5, 'rfe'], [3, 3.2408593602559863, 0.2472966159362192, 0.48286206358121864, 0.15614031216505808, 0.75, 'pca'], [5, 2.602387397756025, 0.37155181204758414, 0.3882530618903668, 0.3328079168300429, 0.5, 'rfe'], [2, 2.3680020967332878, 0.17453293733070402, 0.39476727085158336, 0.18662014176974878, 0.25, 'pca'], [2, 1.1116069412579386, 0.2545955924503448, 0.461039390211762, 0.2799799959644911, 0.5, 'rfe'], [1, 4.849045338733745, 0.3612560143191751, 0.16836383405441807, 0.24326086678781003, 0.75, 'rfe'], [3, 0.1921271323

In [39]:
wtf

          fun: -1.0
            x: [-1, 23, 58, 0.4248674915101974, 0.291990868950023, 0.5, 'rfe']
    func_vals: [-9.997e-01 -1.000e+00 -9.819e-01 -9.994e-01 -1.000e+00
                -1.000e+00 -9.985e-01 -9.946e-01 -1.000e+00 -9.896e-01]
      x_iters: [[3, 29, 89, 0.4389006955136503, 0.349425478714389, 0.5, 'pca'], [-1, 23, 58, 0.4248674915101974, 0.291990868950023, 0.5, 'rfe'], [1, 27, 49, 0.48286206358121864, 0.15614031216505808, 0.75, 'pca'], [5, 26, 74, 0.3882530618903668, 0.3328079168300429, 0.5, 'rfe'], [0, 25, 35, 0.39476727085158336, 0.18662014176974878, 0.25, 'pca'], [0, 22, 51, 0.461039390211762, 0.2799799959644911, 0.5, 'rfe'], [0, 31, 72, 0.16836383405441807, 0.24326086678781003, 0.75, 'rfe'], [1, 20, 71, 0.48357970744980827, 0.3611161268021964, 0.5, 'rfe'], [3, 25, 58, 0.3494040404527473, 0.23520304593555674, 0.75, 'pca'], [4, 30, 73, 0.10542865424484393, 0.34913843821866786, 0.75, 'rfe']]
       models: [GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, 

In [8]:
model_params = {
    'n_estimators': 100,
    'learning_rate': 0.1
}

kwargs = {
    'learning_rate': 0.01,
    'max_depth': 10
}

new_model = model_class(**model_params)

for param, value in kwargs.items():
    setattr(new_model, param, value)

print(new_model.get_params()['learning_rate'])

0.01


In [6]:
can_we_transform_your_features = True
model = cat_clf

extractor = ModelParameterExtractor()
model_params, model_class = extractor.get_model_params_and_class(model) #TODO: revisar
model_type = ModelParameterExtractor.get_package_name(model_class)

initial_feature_space = FeatureSpaceConstants.get_feature_space(model_type, can_we_transform_your_features)
adjusted_feature_space = FeatureSpaceConstants.adjust_search_space(initial_feature_space, model_params)
if can_we_transform_your_features:
    features_percentage_to_use_list = list([cat for cat in FeatureSpaceConstants.CONSTANTS 
                                        if cat.name == 'features_percentage_to_use'][0].categories)
    print(features_percentage_to_use_list)
    data_transformer = DataTransformer(estimator=model_class, X_train=X_train, y_train=y_train, X_eval= Xpercentages=features_percentage_to_use_list, verbose=0)

Adjusting min_data_in_leaf to new range: [1, 50]
[0.25, 0.5, 0.75]


TypeError: DataTransformer.__init__() missing 1 required positional argument: 'X_eval'

In [4]:
mcr(xgb_clf, X_train, y_train, X_train, y_train, True)

Adjusting gamma to new range: [0, 5]
[0.25, 0.5, 0.75]
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa
aaa


          fun: 7
            x: [2, 2.3680020967332878, 0.17453293733070402, 0.39476727085158336, 0.18662014176974878, 0.25, 'pca']
    func_vals: [15 30 ...  7  7]
      x_iters: [[4, 4.221328742905087, 0.4431782470491028, 0.4389006955136503, 0.349425478714389, 0.5, 'pca'], [1, 1.3632814729005662, 0.29106604692854, 0.4248674915101974, 0.291990868950023, 0.5, 'none'], [3, 3.2408593602559863, 0.2472966159362192, 0.48286206358121864, 0.15614031216505808, 0.75, 'rfe'], [5, 2.602387397756025, 0.37155181204758414, 0.3882530618903668, 0.3328079168300429, 0.5, 'none'], [2, 2.3680020967332878, 0.17453293733070402, 0.39476727085158336, 0.18662014176974878, 0.25, 'pca'], [2, 1.1116069412579386, 0.2545955924503448, 0.461039390211762, 0.2799799959644911, 0.5, 'none'], [1, 4.849045338733745, 0.3612560143191751, 0.16836383405441807, 0.24326086678781003, 0.75, 'rfe'], [3, 0.19212713236367365, 0.3537096231829341, 0.48357970744980827, 0.3611161268021964, 0.5, 'none'], [4, 2.0718429411318446, 0.28987900

In [23]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

def precalculate_pca(X_train):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)
    pca = PCA().fit(X_scaled)
    return pca, scaler

def apply_rfe_transformation(X, rfe, percentage):
    ranking = rfe.ranking_
    n_features_to_select = int(len(ranking) * (percentage / 100.0))
    selected_features = np.argsort(ranking)[:n_features_to_select]
    return X[:, selected_features]

def precalculate_rfe(X_train, y_train, estimator):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)
    # Ajusta RFE para el 25% de las características. Puedes ajustar para más y recortar después si es necesario
    n_features_25 = int(X_train.shape[1] * 0.25)
    rfe = RFE(estimator=estimator, n_features_to_select=n_features_25)
    rfe.fit(X_scaled, y_train)
    return rfe, scaler

# Ejemplo de cómo llamar a las funciones
estimator = RandomForestClassifier(n_estimators=100, random_state=42)
pca, pca_scaler = precalculate_pca(X_train)
rfe, rfe_scaler = precalculate_rfe(X_train, y_train, estimator)

In [ ]:
def apply_transformation(X, transformer, scaler, method, percentage):
    X_scaled = scaler.transform(X)
    if method == 'pca':
        n_components = int(len(transformer.explained_variance_ratio_) * (percentage / 100.0))
        return transformer.transform(X_scaled)[:, :n_components]
    elif method == 'rfe':
        # Aplica la transformación RFE. Nota: RFE ya ha sido ajustado al 25%
        # Para 50% y 75%, necesitarías determinar cuáles características quedan basado en el ranking
        support_ = transformer.get_support()
        return X_scaled[:, support_]
    else:
        # No se aplica transformación
        return X_scaled



In [ ]:
transformed_X_test = apply_transformation(X_test, pca if best_method == 'pca' else rfe, 
                                          pca_scaler if best_method == 'pca' else rfe_scaler, 
                                          best_method, best_percentage)

# Para devolver el transformador, puedes crear una función o lógica similar
# que seleccione y configure el transformador basado en 'best_method' y 'best_percentage'
def get_best_transformer(best_method, best_percentage):
    if best_method == 'pca':
        return Pipeline([('scaler', pca_scaler), ('pca', PCA(n_components=int(len(pca.explained_variance_ratio_) * (best_percentage / 100.0))))])
    elif best_method == 'rfe':
        # Aquí deberías determinar cómo manejar RFE si necesitas aplicarlo de nuevo o devolver un transformador preajustado
        pass
    else:
        # Si no se requiere transformación, devuelve solo el escalador
        return pca_scaler

In [37]:
@use_named_args(adjusted_feature_space)
def _objective(**kwargs):
    dim_red = kwargs.get('dimensionality_reduction')
    if dim_red is not None:
        # Llamar al constructor de una clase auxiliar o algo similar que almacene los 6 DFs ya que serán usados múltiples veces. [RFE_75, RFE_50, RFE_25, PCA_75, PCA_50, PCA_25]
        # Solo quiero llamar una vez a RFE() y otra vez a PCA() para generar los datasets. 
        # data = dataset_deck.get_data(dim_red, kwargs.get("features_percentage_to_use"))

    elif dim_red == 'rfe':
        estimator = RandomForestClassifier(n_estimators=100)
        transformer = RFE(estimator, n_features_to_select=kwargs.get("features_percentage_to_use"), step=1)
    else:  # 'none'
        transformer = None
    
    if transformer:
        X_transformed = transformer.fit_transform(X_train, y_train)
    else:
        X_transformed = X_train
    if kwargs.get('dimensionality_reduction') == 'pca':
        print("asd")
    return 32



In [38]:
res = gp_minimize(_objective, FEATURE_SPACE, n_calls=50, random_state=0)

asd
asd
asd
asd
asd
asd
asd
asd
asd
asd
asd
asd
asd
asd
asd
asd
asd
asd
asd


In [23]:
res

          fun: 32
            x: [4, 4, 0.4431782470491028, 0.4389006955136503, 0.349425478714389, 50, 'pca']
    func_vals: [32 32 ... 32 32]
      x_iters: [[4, 4, 0.4431782470491028, 0.4389006955136503, 0.349425478714389, 50, 'pca'], [1, 1, 0.29106604692854, 0.4248674915101974, 0.291990868950023, 50, 'none'], [3, 3, 0.2472966159362192, 0.48286206358121864, 0.15614031216505808, 75, 'rfe'], [5, 3, 0.37155181204758414, 0.3882530618903668, 0.3328079168300429, 50, 'none'], [2, 2, 0.17453293733070402, 0.39476727085158336, 0.18662014176974878, 25, 'pca'], [2, 1, 0.2545955924503448, 0.461039390211762, 0.2799799959644911, 50, 'none'], [1, 5, 0.3612560143191751, 0.16836383405441807, 0.24326086678781003, 75, 'rfe'], [3, 0, 0.3537096231829341, 0.48357970744980827, 0.3611161268021964, 50, 'none'], [4, 2, 0.2898790009153652, 0.3494040404527473, 0.23520304593555674, 75, 'pca'], [5, 5, 0.36501074678001777, 0.10542865424484393, 0.34913843821866786, 75, 'none'], [6, 0, 0.1, 0.1, 0.5, 25, 'rfe'], [1, 

In [ ]:
@use_named_args(FEATURE_SPACE)
def _objective(max_depth, learning_rate, n_features, dimensionality_reduction):
    if dimensionality_reduction == 'pca':
        transformer = PCA(n_components=n_features)
    elif dimensionality_reduction == 'rfe':
        estimator = RandomForestClassifier(n_estimators=100)
        transformer = RFE(estimator, n_features_to_select=n_features, step=1)
    else:  # 'none'
        transformer = None
    
    if transformer:
        X_transformed = transformer.fit_transform(X_train, y_train)
    else:
        X_transformed = X_train

    lgb_data = lgb.Dataset(X_transformed, label=y_train)

    params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'verbose': -1
    }

    cv_results = lgb.cv(params, lgb_data, num_boost_round=1000, nfold=5, 
                        early_stopping_rounds=50, stratified=False, seed=42)

    mean_auc = np.mean(cv_results['auc-mean'])

    complexity = (learning_rate * max_depth) / X_transformed.shape[1] #TODO

    penalizacion = complexity / complexity
    pen_auc = mean_auc + penalizacion
    
    return -pen_auc

In [6]:
import numpy as np
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from skopt import gp_minimize
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE


class mcr:
    def __init__(self, 
                 reduce_number_of_features = True,
                 transform_features = True):
        self.reduce_number_of_features = reduce_number_of_features
        self.transform_features = transform_features

    def _get_model_params(self, input_model):
        if isinstance(input_model, (lgb.LGBMClassifier, 
                                    lgb.LGBMRegressor, )
                                    ):
            model_params = input_model.get_params()
        elif isinstance(input_model, (xgb.XGBClassifier, xgb.XGBRegressor)):
            model_params = input_model.get_params()
        elif isinstance(input_model, lgb.Booster):
            model_params = input_model.params
        elif isinstance(input_model, xgb.Booster): 
            model_params = input_model.attributes()
        else:
            raise ValueError("Model type not supported")
        return model_params
    
    def _adjust_search_space(space, model_params):
        adjusted_space = []
        for dimension in space:
            if dimension.name in model_params:
                model_param_value = float(model_params[dimension.name])
                if model_param_value < dimension.low:
                    print(f"{dimension.name} fixed at {model_param_value} (below search space).")
                    continue

                if model_param_value < dimension.high:
                    print(f"Adjusting upper limit of {dimension.name} to {model_param_value}.")
                    if isinstance(dimension, Integer):
                        adjusted_dimension = Integer(dimension.low, int(model_param_value), name=dimension.name)
                    elif isinstance(dimension, Real):
                        adjusted_dimension = Real(dimension.low, model_param_value, name=dimension.name)
                    adjusted_space.append(adjusted_dimension)
                else:
                    adjusted_space.append(dimension)
            else:
                adjusted_space.append(dimension)
        return adjusted_space
    
    @use_named_args(FEATURE_SPACE)
    def _objective(max_depth, learning_rate, n_features, dimensionality_reduction):
        if dimensionality_reduction == 'pca':
            transformer = PCA(n_components=n_features)
        elif dimensionality_reduction == 'rfe':
            estimator = RandomForestClassifier(n_estimators=100)
            transformer = RFE(estimator, n_features_to_select=n_features, step=1)
        else:  # 'none'
            transformer = None
        
        if transformer:
            X_transformed = transformer.fit_transform(X_train, y_train)
        else:
            X_transformed = X_train

        lgb_data = lgb.Dataset(X_transformed, label=y_train)

        params = {
            'objective': 'binary',
            'metric': 'auc',
            'learning_rate': learning_rate,
            'max_depth': max_depth,
            'verbose': -1
        }

        cv_results = lgb.cv(params, lgb_data, num_boost_round=1000, nfold=5, 
                            early_stopping_rounds=50, stratified=False, seed=42)

        mean_auc = np.mean(cv_results['auc-mean'])

        complexity = (learning_rate * max_depth) / X_transformed.shape[1] #TODO

        penalizacion = complexity / complexity
        pen_auc = mean_auc + penalizacion
        
        return -pen_auc
    
    
    

In [ ]:
def optimize_and_train(X_train, y_train, X_test, y_test, target_name, input_model=None):
    # Inicializa el espacio de búsqueda con rangos por defecto
    space = [
        Integer(3, 10, name='max_depth'),
        Real(0.01, 0.3, name='learning_rate'),
        Integer(10, X_train.shape[1], name='n_features'),
        Categorical(['pca', 'rfe', 'none'], name='dimensionality_reduction'),
    ]

In [ ]:
    model = lgb.LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)
    model.fit(X_train, y_train)
    
    preds = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, preds)
    
    # Número de hojas
    num_leaves = 2 ** max_depth  # Simplificación

    
    return -auc_penalizado

In [ ]:
import numpy as np
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE





def optimize_and_train(X_train, y_train, X_test, y_test, target_name):
    space = [
        Integer(3, 10, name='max_depth'),
        Real(0.01, 0.3, name='learning_rate'),
        Integer(10, X_train.shape[1], name='n_features'),
        Categorical(['pca', 'rfe', 'none'], name='dimensionality_reduction'),
    ]



    # Realizar optimización bayesiana
    res = gp_minimize(objective, space, n_calls=50, random_state=0)

    # Entrenar el modelo final con los mejores hiperparámetros
    best_params = res.x
    max_depth, learning_rate, n_features, dimensionality_reduction = best_params

    if dimensionality_reduction == 'pca':
        final_transformer = PCA(n_components=n_features)
    elif dimensionality_reduction == 'rfe':
        final_estimator = RandomForestClassifier(n_estimators=100)
        final_transformer = RFE(final_estimator, n_features_to_select=n_features, step=1)
    else:
        final_transformer = None
    
    if final_transformer:
        X_final_transformed = final_transformer.fit_transform(X_train, y_train)
    else:
        X_final_transformed = X_train

    final_model = lgb.LGBMClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=best_num_boost_rounds  # Asumiendo que esta variable está accesible o ajustada apropiadamente
    )

    final_model.fit(X_final_transformed, y_train)

    # Devuelve el modelo entrenado y el transformador (si se aplica)
    return final_model, final_transformer

# Suponiendo que X, y son tus datos completos y 'target' es el nombre de tu columna objetivo
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# final_model, final_transformer = optimize_and_train(X_train, y_train, X_test, y_test, 'target')
